Good question — this is exactly the context you need before appreciating **why Runnables were introduced in LangChain**. Let’s go step by step like proper class notes.

---

1. Situation Before Runnables

In earlier versions of LangChain (before late 2023), different components had **different interfaces**:

* `LLM` objects had `.predict()` or `.generate()`
* `ChatModels` had `.predict_messages()` or `.predict()`
* `PromptTemplate` used `.format()`
* `OutputParsers` had `.parse()`
* Retrievers used `.get_relevant_documents()`
* Tools had `.run()` or `.arun()`

Each component worked differently, with **no single standard method** to call them.

---

2. Problems Caused

a) **Inconsistency**
You had to remember many different method names (predict, generate, run, format, parse…). This made code harder to read, write, and maintain.

b) **Difficult Composition**
If you wanted to connect a retriever → prompt template → LLM → parser:

* Each step had its own API.
* You had to manually write “glue code” to pass outputs from one component into the next.

c) **Async vs Sync Confusion**
Some methods supported async (`arun`, `apredict`), some didn’t. Developers had to handle this manually, leading to messy code.

d) **No Built-in Batching/Streaming**

* If you wanted to run a model on a list of inputs, you had to loop yourself.
* If you wanted streaming outputs, each model had its own streaming API (if any). Nothing unified.

e) **Debugging / Observability Pain**
Because there was no standard interface, logging, tracing, or monitoring required writing wrappers around every different component.

---

3. Why Standardization with Runnables

LangChain created the **Runnable Protocol** to fix these issues:

• One method (`.invoke`) to call any component (LLM, retriever, parser, tool, lambda, etc.).
• Same API for synchronous, asynchronous, batched, and streaming execution.
• Easy to compose: use `|` to chain steps instead of manually writing glue code.
• Built-in hooks for observability (logging, tracing, callbacks).

---

4. Real-life Analogy

Imagine before Runnables:

* Every electrical appliance in your house had a **different plug** (TV plug, fan plug, fridge plug).
* You’d need adapters for each one.
* Messy, frustrating, and easy to mess up.

Runnables = **standardized universal plug**.
Now you just plug anything into the same socket, and it works the same way.

---

5. Summary

Before Runnables:

* Many inconsistent method names
* Hard to compose chains
* Async/batch/stream not unified
* Debugging and monitoring messy

After Runnables:

* Single clean API (`invoke`, `batch`, `stream`)
* Easy composition (`|`, `RunnableParallel`, `RunnableBranch`)
* Unified async/batch/stream handling
* Standardized observability

That’s why Runnables are the foundation of modern LangChain.

---

Would you like me to make a **side-by-side code comparison** (before vs after Runnables) so you see the difference clearly?


## **how they standardized**

Perfect — let’s focus on **intuition**. Imagine I’m teaching this in class: *“Why Runnables feel natural and how they solve the old mess.”*

---

1. The Old World (Before Runnables) – Intuition

Think of LangChain like a city before a good transport system:

* Each street had its **own type of bus stop** (different shapes, signs, rules).
* A bus from one street could not easily connect to another — you had to walk and find a new stop.
* Some stops allowed **express buses** (async), some didn’t.
* Some stops only allowed **one passenger at a time** (no batching).

So, if you wanted to travel across the city (retriever → prompt → LLM → parser), you had to constantly switch stops, figure out the rules, and adapt. Messy and exhausting.

---

2. What Runnables Do – Intuitive Fix

Now imagine the city builds a **unified metro system**:

* Every station looks and works the same.
* You tap your metro card once, and you can travel anywhere without worrying about local rules.
* The same card works for single trips (invoke), group trips (batch), or continuous rides (stream).
* You can connect stations in a line (sequence) or split into multiple lines at once (parallel).

That’s exactly what Runnables did for LangChain.

---

3. Step-by-Step Intuition

a) **Single Door (invoke)**
Instead of remembering 10 different doors (`predict`, `run`, `format`…), now everything has one door: `.invoke(input)`.

b) **Group Entry (batch)**
Earlier, if you had 100 inputs, you had to push them one by one. Now, `.batch([...])` lets everyone enter at once.

c) **Live Tour Guide (stream)**
Before, you had to wait till the end to see results. Now, with `.stream()`, you hear updates as they happen (like a guide narrating live during the ride).

d) **Pipe System (| operator)**
Instead of manually handing over papers from one worker to another, you now have a conveyor belt: `retriever | prompt | LLM | parser`. Outputs automatically become inputs for the next.

e) **Traffic Junctions (parallel & branch)**

* Parallel = send the same car to multiple roads at once.
* Branch = traffic cop directs cars to different roads based on rules.

---

4. Why It Feels Natural

• One Mental Model: Instead of learning 10 APIs, you just learn “invoke, batch, stream, pipe.”
• Lego-like Composition: Every piece fits into every other piece.
• Less Friction: You focus on *what you want to build*, not on *how to connect pieces*.

---

5. Intuitive Summary

Before Runnables → messy city with different bus stops and rules.
After Runnables → clean metro system with one card, one interface, multiple options.

So intuitively: **Runnables give you one universal plug (invoke) and a conveyor system (|) to connect everything seamlessly.**

---

Do you want me to now **show this intuition with a before-and-after code example** (like “old way” vs “Runnable way”) so it clicks both conceptually and practically?


In [3]:
from abc import ABC,abstractmethod

class Runnable(ABC):
    
    @abstractmethod
    def invoke(input_data):
        pass

In [9]:
import random
class LLM(Runnable):
    def __init__(self):
        print("model instanciated")

    def invoke(self,prompt):
        sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Python is a versatile programming language.",
    "Lists are useful for storing collections of items.",
    "This is another example sentence."
    ]
        return {'response':random.choice(sentences)}

    def predict(self,prompt):
       """ this method is going to be depricated soon """
       sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Python is a versatile programming language.",
    "Lists are useful for storing collections of items.",
    "This is another example sentence."
    ]
       return {'response':random.choice(sentences)}

In [10]:
model = LLM()

model instanciated


In [11]:
model.invoke("who is the director of the rgukt")

{'response': 'Lists are useful for storing collections of items.'}

In [16]:
class PromptTemplate(Runnable):
    def __init__(self,template,input_varibles):
        self.template = template
        self.input_varibles = input_varibles

    def invoke(self,input_dict):
        return self.template.format(**input_dict)
    def format(self,input_dict):
        """This method is going to be depricated"""
        return self.template.format(**input_dict)
    
        

In [17]:
template = PromptTemplate(
    template= "tell me about the {topic} in a {mode}",
    input_varibles= ['topic','mode']
)

In [18]:
prompt = template.format({'topic':'rgukt','mode':'cool'})

In [19]:
prompt

'tell me about the rgukt in a cool'

In [20]:
model.predict(prompt= prompt)

{'response': 'This is another example sentence.'}

In [22]:
class NakliStrOutputParser(Runnable):

  def __init__(self):
    pass

  def invoke(self, input_data):
    return input_data['response']

In [27]:
class RunnableConnector(Runnable):
    def __init__(self,runnable_list):
        self.runnable_list = runnable_list
        
    def invoke(self,input_data):
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)
        return input_data

In [33]:
template = PromptTemplate(
    template= "write a {length} poem about the {topic}",
    input_varibles= ['length','topic']
)

In [34]:
model = LLM()

model instanciated


In [35]:
chain = RunnableConnector([template , model] )

In [36]:
chain.invoke({'topic':"rgukt",'length':100})

{'response': 'The quick brown fox jumps over the lazy dog.'}